In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from scipy.spatial import cKDTree

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('max_colwidth', 50)

In [3]:
df_path = 'D:\Simon\Documents\GP\Raw_Data\selected_bird_species_with_grids_50km.csv'
df = pd.read_csv(df_path, index_col=0)

C:\Users\simon\AppData\Local\Temp\ipykernel_8220\2799499926.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, index_col=0)


In [4]:
clc_path = 'D:\\Simon\\Documents\GP\\feature_generation_info_files\\U2018_CLC2018_V2020_20u1_shp\\U2018_CLC2018_V2020_20u1.shp'
clc = gpd.read_file(clc_path)

In [5]:
clc.head(3)

,OBJECTID,Code_18,Remark,Area_Ha,ID,Shape_Leng,Shape_Area,geometry
0,341326,111,NaN,28.908170,EU_341326,2464.258292,289081.698549,"POLYGON ((4.65404 43.80421, 4.65492 43.80702, ..."
1,341327,111,NaN,34.903548,EU_341327,2612.021123,349035.483249,"POLYGON ((4.64857 43.80864, 4.64914 43.80790, ..."
2,341328,111,NaN,56.419745,EU_341328,3042.530620,564197.452800,"POLYGON ((5.04233 43.83631, 5.04287 43.83460, ..."


In [6]:
df.head(3)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,-1.051010,0.0,NaN,37718.0,de,50kmE4200N3300
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,0.760781,2.0,NaN,37803.0,de,50kmE4250N3300
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,270.831300,2.0,NaN,39627.0,de,50kmE4450N3050


In [9]:
geometry = [Point(lon, lat) for lon, lat in zip(df['coord_lon'], df['coord_lat'])]
gdf_df = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Perform a spatial join
merged_gdf = gpd.sjoin(gdf_df, clc, how="left", predicate="within")

merged_gdf.drop(columns=['geometry','index_right', 'OBJECTID', 'Remark', 'Area_Ha', 'ID', 'Shape_Leng', 'Shape_Area'], axis=1, inplace=True)

In [10]:
merged_gdf

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id,Code_18
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,-1.051010,0.0,NaN,37718.0,de,50kmE4200N3300,512
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,0.760781,2.0,NaN,37803.0,de,50kmE4250N3300,231
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,270.831300,2.0,NaN,39627.0,de,50kmE4450N3050,121
3,29666414,8.0,Haubentaucher,2018-01-01,NaN,51.076006,11.038316,place,158.941010,8.0,NaN,38301.0,de,50kmE4350N3100,512
4,29656211,8.0,Haubentaucher,2018-01-01,NaN,51.389380,7.067282,place,52.362160,10.0,NaN,108167.0,de,50kmE4100N3100,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2660038,27523548,469.0,Bergpieper,2022-08-01,09:35:04,46.563896,8.551648,precise,2099.035000,4.0,0,11482.0,ch,50kmE4200N2600,321
2660039,27523686,338.0,Mittelspecht,2022-10-08,09:17:16,47.383318,7.666533,precise,802.159700,1.0,0,11482.0,ch,50kmE4100N2650,242
2660040,15002272,123.0,Bergente,2018-02-17,NaN,47.512154,9.436332,precise,391.370000,1.0,0,11245.0,ch,50kmE4250N2700,512
2660041,15002282,8.0,Haubentaucher,2018-02-17,NaN,47.512154,9.436332,precise,391.370000,NaN,0,11245.0,ch,50kmE4250N2700,512
